In [20]:
!pip install tensorflow==2.16.1
import tensorflow as tf
from matplotlib import pyplot as plt
import zipfile
from shutil import copyfile
from time import time
import numpy as np
import os
import random
import nibabel as nib


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [21]:
def read_single_mri(path):
    mri = nib.load(path)
    mri = mri.get_fdata()
    return mri


def find_file_names(directory):
    filenames = os.listdir(directory)
    filenames = [n for n in filenames if '._' not in n]
    return filenames

def resize_label(image):
    new_image = np.zeros((128, 128, 128))
    new_image[:, :, :] = image[56:184, 56:184, 13:141]
    return new_image

def resize_image(image):
    new_image = np.zeros((128, 128, 128, 4))
    new_image[:, :, :, 0] = image[56:184, 56:184, 13:141, 0]
    new_image[:, :, :, 1] = image[56:184, 56:184, 13:141, 1]
    new_image[:, :, :, 2] = image[56:184, 56:184, 13:141, 2]
    new_image[:, :, :, 3] = image[56:184, 56:184, 13:141, 3]
    return new_image

In [22]:
def calculate_tumor_ratio_2d(images):
   
    unique_values = np.zeros(4)
    for im in images:
        unique_values_img = np.unique(im, return_counts=True)[1]
        if len(unique_values_img) == 3:
            unique_values_img = np.append(unique_values_img, 0)
        if len(unique_values_img) == 2:
            unique_values_img = np.append(unique_values_img, np.zeros(2))
        if len(unique_values_img) == 1:
            unique_values_img = np.append(unique_values_img, np.zeros(3))
        unique_values = unique_values + unique_values_img

    print(unique_values / sum(unique_values))

In [23]:
def reduce_umbalance(images, labels):
    remove_indexes = []
    for i in range(0, len(images)):
        unique_values_img = np.unique(labels[i], return_counts=True)[1]

        if len(unique_values_img) == 1:
            remove_indexes.append(i)
        elif sum((unique_values_img / sum(unique_values_img))[1:]) < 0.05:
            remove_indexes.append(i)
    return remove_indexes

In [24]:
path = 'destination_folder/Task01_BrainTumour/imagesTr'
labels_path = 'destination_folder/Task01_BrainTumour/labelsTr'
file_names = find_file_names(path)

random.shuffle(file_names)

path_list = []
path_label_list = []
for filename in file_names:
    path_list.append(os.path.join(path, filename))
    path_label_list.append(os.path.join(labels_path, filename))

train_labels = np.array([resize_label(read_single_mri(path)) for path in path_label_list[0:109]])
#validation_labels = np.array([resize_label(read_single_mri(path)) for path in path_label_list[100:125]])
train_imgs = np.array([resize_image(read_single_mri(path)) for path in path_list[0:109]])
#validation_imgs = np.array([resize_image(read_single_mri(path)) for path in path_list[100:125]])
test_labels = np.array([resize_label(read_single_mri(path)) for path in path_label_list[110:120]])
test_imgs = np.array([resize_image(read_single_mri(path)) for path in path_list[110:120]])

In [25]:
train_imgs_2D = []
train_labels_2D = []


for i in range(0, train_labels.shape[0]):
    image_3d = train_imgs[i]
    label_3d = train_labels[i]

    for j in range(0, image_3d.shape[0]):
        label_slice = label_3d[j, :, :]
        tmp = []
        for z in range(0, 4):
            mri_slice = image_3d[j, :, :, z]
            tmp.append(mri_slice)

        train_imgs_2D.append(tmp)
        train_labels_2D.append(label_slice)
    for j in range(0, image_3d.shape[0]):
        label_slice = label_3d[:, j, :]
        tmp = []
        for z in range(0, 4):
            mri_slice = image_3d[:, j, :, z]
            tmp.append(mri_slice)

        train_imgs_2D.append(tmp)
        train_labels_2D.append(label_slice)
    for j in range(0, image_3d.shape[0]):
        label_slice = label_3d[:, :, j]
        tmp = []
        for z in range(0, 4):
            mri_slice = image_3d[:, :, j, z]
            tmp.append(mri_slice)

        train_imgs_2D.append(tmp)
        train_labels_2D.append(label_slice)

In [26]:
test_imgs_2D = []
test_labels_2D = []

for i in range(0, test_labels.shape[0]):
    image_3d = test_imgs[i]
    label_3d = test_labels[i]

    for j in range(0, image_3d.shape[0]):
        label_slice = label_3d[j, :, :]
        tmp = []
        for z in range(0, 4):
            mri_slice = image_3d[j, :, :, z]
            tmp.append(mri_slice)

        test_imgs_2D.append(tmp)
        test_labels_2D.append(label_slice)
    for j in range(0, image_3d.shape[0]):
        label_slice = label_3d[:, j, :]
        tmp = []
        for z in range(0, 4):
            mri_slice = image_3d[:, j, :, z]
            tmp.append(mri_slice)

        test_imgs_2D.append(tmp)
        test_labels_2D.append(label_slice)
    for j in range(0, image_3d.shape[0]):
        label_slice = label_3d[:, :, j]
        tmp = []
        for z in range(0, 4):
            mri_slice = image_3d[:, :, j, z]
            tmp.append(mri_slice)

        test_imgs_2D.append(tmp)
        test_labels_2D.append(label_slice)

In [27]:
calculate_tumor_ratio_2d(train_labels_2D)


[0.95725152 0.02654215 0.00784103 0.0083653 ]


In [28]:
indexes = reduce_umbalance(train_imgs_2D, train_labels_2D)

for ind in sorted(indexes, reverse=True):
    del train_imgs_2D[ind]
    del train_labels_2D[ind]

calculate_tumor_ratio_2d(train_labels_2D)

[0.88075193 0.07128971 0.02314808 0.02481028]


In [29]:
removing_indexes = []
for i in range(0, len(train_labels_2D)):
    if (i % 3) != 0:
        removing_indexes.append(i)
        
print(len(removing_indexes))

for ind in sorted(removing_indexes, reverse=True):
    del train_imgs_2D[ind]
    del train_labels_2D[ind]

8982


In [30]:
print(len(train_labels_2D))

4491


In [31]:
X = np.array(train_imgs_2D)
Y = np.array(train_labels_2D)

In [32]:
N = len(X)
train_index = int(0.8*N)
X_train = X[:train_index]
Y_train = Y[:train_index]
X_validation = X[train_index:]
Y_validation = Y[train_index:]

In [33]:
def mask_multi_class(mask):
  multi_mask = tf.one_hot(tf.cast(mask, tf.int32), depth=4, axis=0)
  return multi_mask

In [34]:
X_train = np.moveaxis(X_train, [1,3], [3,1])
X_validation = np.moveaxis(X_validation, [1,3], [3,1])
X_train.shape

(3592, 128, 128, 4)

In [35]:
Y_train_multi = np.zeros((train_index,4,128,128))
Y_validation_multi = np.zeros((N-train_index,4,128,128))
Y_test_multi = np.zeros((len(test_labels_2D),4,128,128))
for i in range(0,len(Y_train)):
  Y_train_multi[i] = mask_multi_class(Y_train[i])

for i in range(0,len(Y_validation)):
  Y_validation_multi[i] = mask_multi_class(Y_validation[i])
  
for i in range(0, len(test_labels_2D)):
    Y_test_multi[i] = mask_multi_class(test_labels_2D[i])

Y_train_multi = np.moveaxis(Y_train_multi, [1,3], [3,1])
Y_validation_multi = np.moveaxis(Y_validation_multi, [1,3], [3,1])
Y_train_multi.shape

(3592, 128, 128, 4)

In [36]:
with open('train_imgs_2D.npy', 'wb') as f:
    np.save(f, X_train)
with open('validation_imgs_2D.npy', 'wb') as f:
    np.save(f, X_validation)
with open('test_imgs_2D.npy', 'wb') as f:
    np.save(f, np.array(test_imgs_2D))

In [37]:
type(Y_train_multi)

numpy.ndarray

In [38]:
with open('train_labels_2D.npy', 'wb') as f:
    np.save(f, Y_train_multi)
with open('validation_labels_2D.npy', 'wb') as f:
    np.save(f, Y_validation_multi)
with open('test_labels_2D.npy', 'wb') as f:
    np.save(f, np.array(Y_test_multi))